# Model Processing

## Imports & General Settings 

In [ ]:
import unittest
import os
import sys
import pathlib
import urllib
import shutil
import re
import zipfile
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from IPython.display import display
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import posixpath
import wfdb
import pycwt as wavelet
from data import WaveletTransform, AFECGDataset
from PIL import Image
import dsp
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time
import os
from tqdm.notebook import trange, tqdm


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
test = unittest.TestCase()
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

## Dataset creation

In [ ]:
dataset_name = 'afdb'
dataset = AFECGDataset(dataset_name, '../data/files/')

In [ ]:
dataset.load()

In [ ]:
data, label = dataset[0]
data.shape

In [ ]:
images_per_sample = 20
total_data_size = len(dataset)
print("Total data size: ", total_data_size)

In [ ]:
# data = [dataset[i][0] for i in range(total_data_size)]
# labels = [dataset[i][1] for i in range(total_data_size)]

### Example of one ECG sample

In [ ]:
# samples, label = data[0], labels[0]
# print('P-signal: ', samples)
# print('Has AF: ', 'Yes' if label == 1 else 'No')

In [ ]:
# to_wavelet = WaveletTransform(wavelet.Morlet(6), resample=20)
# t = to_wavelet(data[0][0])
# image_test = (t * 100 * 255).int() # Simple visualization test
# transforms.ToPILImage()(image_test).show()

##  Wavelet Transform

In [ ]:
# Total data size is 1397
# You can choose the data size 
# data_size = len(dataset)

In [ ]:
# fmt = '../data/new/sample_{}.pt'
# to_wavelet = WaveletTransform(wavelet.Morlet(6), resample=20)
# start = time.time()

# sample_count = data_size
# transformed_data = []
# # transformed_labels= []

# skip = 0
# print('Preparing {} samples'.format(sample_count))
# for sample_idx, sample in tqdm(enumerate(data[:sample_count])):
#     wavelets = []
#     filepath = fmt.format(sample_idx)
    
#     if os.path.isfile(filepath):
#         # print('Skip {},{}'.format(sample_idx, signal_idx))
#         t = torch.load(fmt.format(sample_idx, signal_idx))
#         skip += 1
#         transformed_data.append(t)
#         continue
#     # print(sample)
#     for signal_idx, signal in enumerate(sample):
#         # print(signal)
#         # filepath = fmt.format(sample_idx, signal_idx)
#         # if os.path.isfile(filepath):
#              # print('Skip {},{}'.format(sample_idx, signal_idx))
#         #     skip += 1
#         #     continue
#         wavelets.append(to_wavelet(signal))
        
#     t = torch.stack(wavelets)
#     t = t.unsqueeze(1)
#     transformed_data.append(t)
    
#     torch.save(t, filepath)
    
# end = time.time()
# print('Elapsed time: {} ms'.format(1000 * (end - start)))
# print('Skipped {} files'.format(skip))

In [ ]:
# transformed_data[0].shape

## Train & Test set creation

In [ ]:
# x_train, x_test, y_train, y_test =  train_test_split(transformed_data, labels[:sample_count], test_size=0.2, random_state=1)

## CNN

In [ ]:
class ConvNet(nn.Module):
    def _calculate_ouput_size(padding, stride, kernel):
        n_in = self.width * self.height
        pass
    
    def __init__(self, size, in_channels=1):
        super(ConvNet, self).__init__()
        self.width, self.height = size
                
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 10, kernel_size=(3,21)),
            nn.ReLU(),
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(10, 10, kernel_size=(3,21)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(10, 10, kernel_size=(4,21)),
            nn.ReLU(),
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(10, 10, kernel_size=(4,21)),
            nn.ReLU(),
        )
        
        self.fc = nn.Linear(2540, 50)
        
    def forward(self, x):        
        out = self.layer1(x)
        # print(out.shape)
        out = self.layer2(out)
        # print(out.shape)
        # out = out.reshape(out.size(0), -1)
        out = self.layer3(out)
        # print(out.shape)
        out = self.layer4(out)
        # print(out.shape)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        # print(out.shape)
        return out

In [ ]:
display(ConvNet((256, 256)))

In [ ]:
# x0 = x_train[0][0].float()
# encoder_cnn = ConvNet((375, 20))

# display(x0.unsqueeze(0).shape)
# h = encoder_cnn(x0.unsqueeze(0))
# print(h.shape)

# test.assertEqual(h.dim(), 2)
# test.assertSequenceEqual(h.shape, (1, 50))

In [ ]:
# model = encoder_cnn
# num_epochs = 100
# total_size = len(x_train)
# test.assertEqual(total_size, len(y_train))

# # Loss and optimizer
# learning_rate = 0.001
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# # Train the model
# total_step = len(x_train)
# loss_list = []
# acc_list = []

# for epoch in range(num_epochs):
#     acc = 0
#     for idx, (samples, label) in enumerate(zip(x_train, y_train)):
#         label = torch.tensor([label]).long()
#         for image in samples:
            
#             # Run the forward pass
#             output = model(image.unsqueeze(0).float())
#             loss = criterion(output, label)
#             loss_list.append(loss.item())
            
#             # Backprop and perform Adam optimisation
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
            
#             # Track the accuracy
#             _, predicted = torch.max(output.data, 1)
#             correct = (predicted == label).sum().item()
#             acc += correct
#             print(correct)
                        
#     acc = acc / (total_size * 20)          
#     acc_list.append(acc)
#     print('Epoch [{}/{}], Accuracy: {:.2f}%'
#           .format(epoch + 1, num_epochs, acc * 100))    

## BRNN

In [ ]:
class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_gates):
        super(BRNN, self).__init__()
        self.bi_rnn = torch.nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_gates,
                                    batch_first=False, bidirectional=True)

    def forward(self, X):
        output, hn = self.bi_rnn(X)
        return output

In [ ]:
display(BRNN(50, 50, images_per_sample))

## Attention

Notations:

* $Y = \left[ y_1, \ldots, y_T \right]$ – the input matrix of size $\left( N \times T \right)$, where $N$ is the number of features in a single output vector of the BRNN

* $w_\mathrm{att}$ – The parameters of the attention model, of size $\left( N \times 1 \right)$, where $N$ is the number of features in a single output vector of the BRNN

* $\alpha$ – The attention weights, given as $\alpha = \mathrm{softmax} \left( w_\mathrm{att}^T Y \right)$. This is an element-wise softmax, where the output size of $\alpha$ is $\left( 1 \times T \right)$

* $h_\mathrm{att}$ – Output of the attention mechanism, given by $h_\mathrm{att} = Y \alpha^T$, of size $\left( N \times 1 \right)$, i.e. a vector of $N$ features.

In [ ]:
class SoftmaxAttention(nn.Module):
    def __init__(self, input_size):
        super(SoftmaxAttention, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(1, input_size))
        torch.nn.init.xavier_uniform_(self.weight)
    
    def forward(self, X):
        # X [T, B, N]
        # weight [N x 1]
        batch_size = X.size(1)
        X = X.transpose(0,1) # (B, T, N)
        alignment_scores = X.matmul(self.weight.t())
        # print('AS: ', alignment_scores)
        # print('wT: ', self.weight.t().shape)
        # print('AS: ', alignment_scores.shape)
        # alpha [T x 1]
        attn_weights = nn.functional.softmax(alignment_scores, dim=0)
        # print('X: ', X.shape)
        # print('ATT: ', attn_weights.shape)
        # print('X: ', X.shape)
        # h_att [1 x N]
        return torch.bmm(attn_weights.transpose(1, 2), X)

## Full model

In [ ]:
class Baseline(nn.Module):
    def __init__(self):
        super(Baseline, self).__init__()
        
        lst = []
        for i in range(images_per_sample):
            conv = ConvNet((375, 20))
            lst.append(conv)
            self.add_module('conv{}'.format(i), conv)
            
        self.cnn_layers = lst
        self.brnn = BRNN(50, 50, images_per_sample)
        self.attention = SoftmaxAttention(100)
        self.fc = nn.Linear(100, 2)


    def forward(self, X): 
        """
        Perform forward propagation on a batch of data
        :param X: A tensor of shape (B, N, W, H) where (W, H) are the image dimensions,
        N is the number of images per sample, and B is the batch size
        """
        X = X / X.min()
        print('input: ', X.shape)
        # print(X[0,0,:].float().unsqueeze(1), X[0,1,:].float().unsqueeze(1))
        out = [self.cnn_layers[i](X[:,i,:].float().unsqueeze(1)) for i in range(images_per_sample)]
        out = torch.stack(out)
        # print('Equal:', out[0][0] == out[0][1])
        # print('After CNN: ', out[0][0], out[0][1])
        out = self.brnn(out)
        print('After BRNN: ', out)
        out = self.attention(out.squeeze(1))
        out = out.squeeze(1)
        print('After ATT: ', out)
        out = self.fc(out)
        out = out.squeeze(1)
        print('After FC: ', out)
        return F.softmax(out)

In [ ]:
# display(Baseline())

### Training 

In [ ]:
# data, label = dataset[0]
# label.unsqueeze(0)

In [ ]:
model = Baseline()
num_epochs = 200
# total_size = len(x_train)
# test.assertEqual(total_size, len(y_train))

# Preparation
dataloader = DataLoader(dataset, batch_size=90)

# Loss and optimizer
learning_rate = 0.001
weight_decay = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Train the model
total_step = total_data_size
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    acc = 0
    total_size = len(dataset)
    for batch_data, batch_labels in tqdm(dataloader):

        print(batch_data.shape)
        
        optimizer.zero_grad()

        # Run the forward pass
        output = model(batch_data)
        
        print(output.shape, batch_labels.shape)
        
        loss = criterion(output, batch_labels)
        
        # Backprop and perform optimisation
        loss_list.append(loss.item())
        loss.backward()
        optimizer.step()
            
        # Track the accuracy
#         probability = torch.distributions.categorical.Categorical(output)
#         prediction = probability.sample()
        prediction = output.argmax(dim=1)
        
        print('Output:', output)
        print('Ground truth:', batch_labels)
        print('Prediction:', prediction)
        correct = (prediction == batch_labels).sum().item()
        print('Correct: {}'.format(correct))
        acc += correct
                    
    acc = acc / (total_size)          
    acc_list.append(acc)
    print('Epoch [{}/{}], Accuracy: {:.2f}%'
          .format(epoch + 1, num_epochs, acc * 100))    